<a href="https://colab.research.google.com/github/emilianodesu/RLA2/blob/main/breakout/breakout_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cloning repository to execute in Google Colab

In [2]:
import os
def detect_colab() -> bool:
    """
    Detect whether the runtime is Google Colab.

    Returns:
        bool: True if running in Google Colab; False otherwise.
    """
    try:
        import google.colab  # type: ignore
        return True
    except ImportError:
        return False

IN_COLAB = detect_colab()
BASE_PATH = '/content/' if IN_COLAB else './'

print(f"IN_COLAB: {IN_COLAB}")
print(f"BASE_PATH: {BASE_PATH}")

if IN_COLAB:
    # Clone Github repo
    if not os.path.exists(f"{BASE_PATH}/RLA2"):
        !git clone https://github.com/emilianodesu/RLA2.git
    os.chdir(f"{BASE_PATH}/RLA2")
    print(f"Current directory: {os.getcwd()}")

IN_COLAB: True
BASE_PATH: /content/
Current directory: /content/RLA2


# Deep Q-Network on Atari games using Custom CNN

This notebook will explore the implementation of a DQN from Stable Baseline3 using a Custom CNN feature extractor on Atari Environment.

### What you will learn?
* Using Deep Q-Network from Stable Baseline3 Using Custom CNN
* Training on Breakout Environment

## Setup
Install necessary dependencies.

In [4]:
# Installing the necessary packages for Atari environments
!pip install gymnasium
!pip install gymnasium[atari]
!pip install ale-py
# Installing Stable Baselines3 with extra dependencies
!pip install "stable-baselines3[extra]"
!pip install moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 7.3 MB/s eta 0:00:00


In [5]:
# Installing necessary packages for visualization and virtual display
!sudo apt-get update
!sudo apt-get install -y cmake
# Updating the package list and installing ffmpeg and freeglut3-dev for visualization, xvfb for virtual display
!sudo apt-get install -y ffmpeg freeglut3-dev xvfb

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,594 kB]
Fetched 1,722 kB in 2s (782 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as reposit

In [6]:
if not IN_COLAB: # only for AWS sagemaker
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # suppress TF INFO/WARN/ERROR

Import libraries

In [7]:
# Import necessary libraries and modules
import gymnasium as gym
import stable_baselines3
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch
from pathlib import Path
import base64
from IPython import display as ipythondisplay

# Import utility functions for creating Atari environments and stacking frames
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

print(f"{gym.__version__=}")
print(f"{stable_baselines3.__version__=}")

gym.__version__='1.2.1'
stable_baselines3.__version__='2.7.0'


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Configure Environment

In [8]:
# Set up a fake display for rendering videos in the cloud environment
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

## Callbacks and directory setup

In [9]:
# Callback for saving the model at regular intervals
class SaveOnIntervalCallback(BaseCallback):
    def __init__(self, save_interval: int, save_path: str, verbose=1):
        super().__init__(verbose)
        self.save_interval = save_interval
        self.save_path = save_path

    def _on_step(self) -> bool:
        if self.num_timesteps % self.save_interval == 0:
            save_file = os.path.join(self.save_path, f'model_{self.num_timesteps}')
            self.model.save(save_file)
            if self.verbose > 0:
                print(f'Saving model to {save_file}.zip')
        return True

In [27]:
# Creating directories for storing logs and models
log_dir = "dqn/logs/"
models_dir = "dqn/models/"

os.makedirs(log_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

## Define Custom Feature Extractor and Create DQN Model

In [11]:
# Custom CNN Feature Extractor
class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 512):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # The observation space shape for Atari is (4, 84, 84) after frame stacking and processing
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute the size of the output of the CNN
        with torch.no_grad():
            n_flatten = self.cnn(
                torch.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.linear(self.cnn(observations))

#### Initial Atari Environment for 'Breakout'

In [12]:
import ale_py
env_id = "BreakoutNoFrameskip-v4"
env = make_atari_env(env_id, n_envs=4, seed=0, monitor_dir=log_dir)
env = VecFrameStack(env, n_stack=4)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


#### Initial the DQN model using custon CNN feature extractor

In [13]:
# Initialize the DQN agent
model = DQN(
    env=env,
    policy='CnnPolicy',  # This is a placeholder, as we will override the feature extractor
    verbose=1,
    learning_rate=0.0001,
    buffer_size=100000, # Buffer size for Atari is typically larger, e.g., 100k or 1M
    learning_starts=100000,
    gradient_steps=1,
    exploration_fraction=0.1,
    exploration_final_eps=0.01,
    train_freq=4,
    batch_size=32,
    # Activate CustomCNN
    policy_kwargs={'features_extractor_class': CustomCNN}
)

Using cpu device
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## Train the Model

In [ ]:
if not IN_COLAB: # Only train in AWS Sagemaker
    # Set up the callback for saving models
    save_interval = 100000
    save_callback = SaveOnIntervalCallback(save_interval, models_dir)

    # Set the total number of training steps
    total_timesteps = 10000000

    # Train the DQN agent
    model.learn(total_timesteps=total_timesteps, callback=save_callback)

    # Save the final trained model
    final_model_path = os.path.join(models_dir, f'model_{total_timesteps}')
    model.save(final_model_path)

----------------------------------
| rollout/            |          |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 4        |
|    fps              | 253      |
|    time_elapsed     | 1        |
|    total_timesteps  | 268      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 8        |
|    fps              | 302      |
|    time_elapsed     | 1        |
|    total_timesteps  | 360      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 12       |
|    fps              | 354      |
|    time_elapsed     | 1        |
|    total_timesteps  | 480      |
----------------------------------
----------------------------------
| rollout/          

In [18]:
x, y = ts2xy(load_results(log_dir), 'timesteps')

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [31]:
load_results(log_dir)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,index,r,l,t


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [32]:
log_dir

'dqn/logs/'

## Performance evaluation

In [29]:
def plot_results(log_folder: str):
    """
    Plots the training curve from the Monitor log file.
    :param log_folder: the save directory of the Monitor logs
    """
    x, y = ts2xy(load_results(log_folder), 'timesteps')

    # Smooth the curve
    y_smooth = np.convolve(y, np.ones(100)/100, mode='valid')
    x_smooth = x[len(x) - len(y_smooth):]

    fig = plt.figure("Training Curve")
    plt.plot(x_smooth, y_smooth, label="Smoothed Reward")
    plt.plot(x, y, alpha=0.2, label="Raw Reward")
    plt.xlabel("Number of Timesteps")
    plt.ylabel("Rewards")
    plt.title("Training Curve")
    plt.legend()
    plt.show()

# Call the function after training to see the learning curve
plot_results(log_dir)

ValueError: v cannot be empty

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## Video Recording and Display Functions

In [ ]:
# Functions to record and show videos of the agent playing

def show_videos(video_path="", prefix=""):
    html = []
    for mp4 in Path(video_path).glob(f"{prefix}*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            f'''<video alt="{mp4.name}" autoplay loop controls style="height: 400px;">
                  <source src="data:video/mp4;base64,{video_b64.decode('ascii')}" type="video/mp4" />
             </video>'''
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

def record_video(env_id, model, video_length=500, prefix="", video_folder="dqn/videos"):
    os.makedirs(video_folder, exist_ok=True)
    eval_env = make_atari_env(env_id, n_envs=1, seed=0, vec_env_cls=DummyVecEnv)
    eval_env = VecFrameStack(eval_env, n_stack=4)
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )
    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs, deterministic=True) # Use deterministic for evaluation
        obs, _, _, _ = eval_env.step(action)
    eval_env.close()

## Record and Display Videos at Different Training Stages

In [ ]:
# Functions for analyzing the trained models

# Function to get the identifiers of saved models
def get_model_identifiers(models_dir):
    files = os.listdir(models_dir)  # Listing files in the models directory
    model_files = [f for f in files if f.startswith('model_')]  # Filtering out model files
    identifiers = [f.split('_')[1] for f in model_files]  # Extracting identifiers from file names
    return identifiers

# Function to find key identifiers (earliest, middle, final)
def find_key_identifiers(identifiers):
    identifiers.sort()  # Sorting identifiers
    earliest = identifiers[0]  # Earliest identifier
    final = identifiers[-1]  # Final identifier
    middle = identifiers[len(identifiers) // 2]  # Middle identifier
    return earliest, middle, final

# Function to view videos of the models at different training stages
def view(models_dir):
    identifiers = get_model_identifiers(models_dir)  # Getting model identifiers
    print(identifiers)
    earliest, middle, final = find_key_identifiers(identifiers)  # Finding key identifiers

    # Recording and displaying videos at the beginning, middle, and end of training
    for stage, identifier in zip(["beginning", "middle", "end"], [earliest, middle, final]):
        model_path = os.path.join(models_dir, f'model_{identifier}')  # Forming the model path
        model = DQN.load(model_path)  # Loading the model
        record_video("BreakoutNoFrameskip-v4", model, video_length=5000, prefix=f'dqn-breakout-{stage}')  # Recording video
        show_videos("dqn/videos/", prefix=f'dqn-breakout-{stage}')  # Showing videos

In [ ]:
# Changes the model directory below
models_dir="dqn/models"
view(models_dir)  # Calling the view function

['100000.zip', '200000.zip', '300000.zip', '400000.zip', '500000.zip', '600000.zip', '700000.zip', '800000.zip', '900000.zip', '1000000.zip', '1100000.zip', '1200000.zip', '1300000.zip', '1400000.zip', '1500000.zip', '1600000.zip', '1700000.zip', '1800000.zip', '1900000.zip', '2000000.zip', '2100000.zip', '2200000.zip', '2300000.zip', '2400000.zip', '2500000.zip', '2600000.zip', '2700000.zip', '2800000.zip', '2900000.zip', '3000000.zip', '3100000.zip', '3200000.zip', '3300000.zip', '3400000.zip', '3500000.zip', '3600000.zip', '3700000.zip', '3800000.zip', '3900000.zip', '4000000.zip', '4100000.zip', '4200000.zip', '4300000.zip', '4400000.zip', '4500000.zip', '4600000.zip', '4700000.zip', '4800000.zip', '4900000.zip', '5000000.zip', '5100000.zip', '5200000.zip', '5300000.zip', '5400000.zip', '5500000.zip', '5600000.zip', '5700000.zip', '5800000.zip', '5900000.zip', '6000000.zip', '6100000.zip', '6200000.zip', '6300000.zip', '6400000.zip', '6500000.zip', '6600000.zip', '6700000.zip', '68

MoviePy - Done !
MoviePy - video ready /home/sagemaker-user/RLA2/breakout/dqn/videos/dqn-breakout-beginning-step-0-to-step-5000.mp4


Saving video to /home/sagemaker-user/RLA2/breakout/dqn/videos/dqn-breakout-middle-step-0-to-step-5000.mp4
MoviePy - Building video /home/sagemaker-user/RLA2/breakout/dqn/videos/dqn-breakout-middle-step-0-to-step-5000.mp4.
MoviePy - Writing video /home/sagemaker-user/RLA2/breakout/dqn/videos/dqn-breakout-middle-step-0-to-step-5000.mp4



MoviePy - Done !
MoviePy - video ready /home/sagemaker-user/RLA2/breakout/dqn/videos/dqn-breakout-middle-step-0-to-step-5000.mp4


Saving video to /home/sagemaker-user/RLA2/breakout/dqn/videos/dqn-breakout-end-step-0-to-step-5000.mp4
MoviePy - Building video /home/sagemaker-user/RLA2/breakout/dqn/videos/dqn-breakout-end-step-0-to-step-5000.mp4.
MoviePy - Writing video /home/sagemaker-user/RLA2/breakout/dqn/videos/dqn-breakout-end-step-0-to-step-5000.mp4



MoviePy - Done !
MoviePy - video ready /home/sagemaker-user/RLA2/breakout/dqn/videos/dqn-breakout-end-step-0-to-step-5000.mp4
